# Import libraries

In [ ]:
# Import libraries
from pymysql import connect
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
np.set_printoptions(threshold=np.inf)
import os
import warnings
warnings.filterwarnings('ignore')
from getpass import getpass
import re

# Connect to database

In [ ]:
IP = getpass("Please enter database IP: ")
User = getpass("Please enter database User: ")
Password = getpass("Please enter database Password: ")
db = connect(host = IP,
             user = User,
             passwd = Password)

# Define cursor object
cur = db.cursor()
print("Successfully connect to database!")

# Define query to see all the database available in the MYSQL server
query = 'show databases'
cur.execute(query)
print("Showing all databases available...", '\n')

# Fetch all the databases
db = cur.fetchall()

# Iterate through all the databases
for data in db:
    data = ''.join(data)
    print(data)
print('\n', "Total databases in the connection: ", len(db))

## Showing all tables in the selected database

In [ ]:
## Getting all databases in the connection
db_data = []
for data in db:
    data = ''.join(data)
    db_data.append(data)
userin = input("Please select the database to be used: ")

while userin not in db_data:
    userin = input("Please enter the database to be used correctly: ")

db = connect(host = IP,
                   user = User,
                   passwd = Password,
                   database = userin)
cur = db.cursor()

# Define query to fetch all table names
query = "SHOW TABLES"
cur.execute(query)

# Fetch all table names
tables = cur.fetchall()
print("Database ", userin, "selected! The tables available in selected database are: ", '\n')

# Iterate through all the tables
for table in tables:
    table = ''.join(table)
    print(table)
print('\n')
print("Total tables in database: ", len(tables))

In [ ]:
# Request whether user want to transform all tables with data into data frame or query on single tables
choice = input("Do you wish to get all tables transformed into data frames? (y/n): ")
while choice.lower() not in ['y', 'n', 'yes', 'no']:
    print('\n')
    choice = input("Please enter your choice correctly! (y/n): ")
    print('\n')
if choice.lower() in ['n', 'no']:
    choice2 = input("Do you want to end the query or continue to explore any table? (y/n): ")
    while choice2.lower() not in ['y', 'n', 'yes', 'no']:
        choice2 = input("Please enter your choice correctly! (y/n): ")
    if choice2.lower() in ['n', 'no']:
        table2 = []
        for tb in tables:
            tb = ''.join(tb)
            table2.append(tb)
        print("Query end!")
    elif choice2.lower() in ['y', 'yes']:
        choice3 = input("Which table you want to query for: ")
        table2 = []
        for tb in tables:
            tb = ''.join(tb)
            table2.append(tb)
        while choice3 not in table2:
            choice3 = input("Table not found! Kindly enter your choice again: ")
        query = "Select * from `{}`".format(choice3)
        df = pd.read_sql(query, db)
        print(df)
        print('\n')
    choice4 = input("Do you want to continue query and display table? (y/n): ")
    while choice4.lower() not in ['y', 'n', 'yes', 'no']:
        choice4 = input("Please enter your choice correctly! (y/n): ")
    if choice4.lower() in ['n', 'no']:
        print('\n')
        print("Query end!")
    while choice4.lower() in ['y', 'yes']:
            choice5 = input("Pelase enter table you want to query for, else press n / no: ")
            if choice5.lower() in ['n', 'no']:
                print('Query end!')
                break
            while choice5 not in table2:
                choice5 = input("Table not found! Kindly enter your choice again: ")
            query = "Select * from `{}`".format(choice5)
            df = pd.read_sql(query, db)
            print(df)

elif choice.lower() in ['y', 'yes']:
    print('\n')
    print("Extracting those tables with data inside...")
    print('\n')
    # Export tables into Data Frames
    dfs = {}
    for table in tables:
        table = ''.join(table[0])
        query = "Select * from `{}`".format(table)
        df = pd.read_sql(query, db)
        if not df.empty: # Check if Data Frame has rows
            dfs[table] = df
    # Access the Data Frame
    for table, df in dfs.items():
        print("Table: {}".format(table))
        print("Shape: {}".format(df.shape))
        print("-----------------------------", '\n')
    print("Total data frames with data inside: ", len(dfs))

In [ ]:
# Request which table user want to view
if choice.lower() in ['y', 'yes']:
    choice12 = input("Do you want to query any table? (y/n) ")
    while choice12.lower() not in ['y', 'n', 'yes', 'no']:
        choice12 = input("Please enter your choice correctly! (y/n): ")
    if choice12.lower() in ['n', 'no']:
        print("Table query session end!")
    if choice12.lower() in ['y', 'yes']:
        keys = list(dfs.keys())
        que = input("Which table to be displayed (One table per query): ")
        while que not in keys:
            print(que, " not valid!", '\n')
            que = input("Please enter table name to be displayed: ")
        print(dfs[que])
        choice7 = input("Do you want to query again? (y/n) ")
        while choice7.lower() not in ['y', 'yes', 'n', 'no']:
            choice7 = input("Invalid response! Kindly enter again: ")
        if choice7.lower() in ['n', 'no']:
            print("Query end! Proceed to next step...")
        while choice7.lower() in ['y', 'yes']:
            que = input("Which table to be displayed (One table per query): ")
            while que not in keys:
                print(que, " not valid!")
                que = input("Please enter table name to be displayed: ")
            print(dfs[que])
            choice7 = input("Do you want to continue display table: (y/n) ")
            while choice7.lower() not in ['y', 'n', 'yes', 'no']:
                choice7 = input("Invalid response! Kindly enter again: ")    
            if choice7.lower() in ['n', 'no']:
                print("Query end! Proceed to next step...")

In [ ]:
# Define fuction to query relationship between tables
def tb_colq(choice11):
    tables2 = []
    for table in tables:
        table = ''.join(table)
        tables2.append(table)
    while choice11 not in tables2:
        print(choice11, " is invalid!")
        choice11 = input("Please enter again: ")
    # Prepare the SQL query with proper placeholders
    query = "SELECT TABLE_NAME, COLUMN_NAME, CONSTRAINT_NAME, REFERENCED_TABLE_NAME, REFERENCED_COLUMN_NAME FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE WHERE REFERENCED_TABLE_SCHEMA = %s AND REFERENCED_TABLE_NAME = %s;"

    # Execute the query with the provided parameters
    cur.execute(query, (userin, choice11))

    # Fetch all the results
    results = cur.fetchall()

    # Print the results
    tb_rel = pd.DataFrame(results, columns = ['TABLE_NAME', 'COLUMN_NAME', 'CONSTRAINT_NAME', 'REFERENCED_TABLE_NAME', 'REFERENCED_COLUMN_NAME'])
    return print(tb_rel)

choice10 = input("Do you want to query for relationship between tables? (y/n) ")
while choice10.lower() not in ['y', 'n', 'yes', 'no']:
    choice10 = input("Please enter your choice correctly! (y/n): ")
if choice10.lower() in ['n', 'no']:
    print("Query end! Proceed to next step...")
else:
    
    # Prompt the user for input
    choice11 = input("Please enter the related table for the query: ")
    print('\n')
    tb_colq(choice11)
    choice12 = input("Do you want to continue checking for other table? (y/n) ")
    while choice12.lower() not in ['y', 'n', 'yes', 'no']:
        choice12 = input("Please enter your choice correctly! (y/n): ")
    if choice12.lower() in ['n', 'no']:
        print('\n')
        print("Query end! Proceed to next step...")
    while choice12.lower() in ['y', 'yes']:
        choice11 = input("Please enter the related table for the query: ")
        print('\n')
        tb_colq(choice11)
        choice12 = input("Do you want to continue checking for other table? (y/n) ")
        while choice12.lower() not in ['y', 'n', 'yes', 'no']:
            choice12 = input("Please enter your choice correctly! (y/n): ")
        if choice12.lower() in ['n', 'no']:
            print("Query end! Proceed to next step...")
            break